In [1]:
import polars as pl

In [2]:
import redis

In [3]:
import time

In [4]:
import pandas as pd

In [5]:
df = pl.scan_parquet("kjoretoyinfo_fra_2020.parquet")

In [6]:
df = pl.read_parquet("kjoretoyinfo_fra_2020.parquet")

In [7]:
df = df.with_columns([
    pl.col("tekn_reg_f_g_n").cast(pl.Utf8).str.strptime(pl.Datetime, "%Y%m%d").alias("tekn_reg_f_g_n")
])

In [8]:
df = df.with_columns([
    pl.col("tekn_neste_pkk").cast(pl.Utf8).str.strptime(pl.Datetime, "%Y%m%d", strict=False).alias("tekn_neste_pkk")
])

In [9]:
df = df.with_columns([
    pl.col("tekn_reg_eier_dato").cast(pl.Utf8).str.strptime(pl.Datetime, "%Y%m%d").alias("tekn_reg_eier_dato")
])

In [10]:
df = df.with_columns([
    ((pl.col("tekn_drivstoff") == "5") | (pl.col("tekn_drivstoff") == "5,5")).alias("elbil")
])

In [11]:
# Anta at df er din Polars DataFrame
selected_columns = [
    "tekn_reg_f_g_n", "tekn_reg_eier_dato", "tekn_aksler_drift", "tekn_merke", 
    "tekn_modell", "tekn_drivstoff", "tekn_neste_pkk", "tekn_farge", "elbil"
]
# Velg spesifikke kolonner
df_selected = df.select(selected_columns)

In [12]:
df_selected

tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_aksler_drift,tekn_merke,tekn_modell,tekn_drivstoff,tekn_neste_pkk,tekn_farge,elbil
datetime[μs],datetime[μs],i64,i64,str,str,datetime[μs],str,bool
2021-10-13 00:00:00,2021-10-13 00:00:00,2,3900,"""VITO""","""2""",2025-10-13 00:00:00,"""05""",false
2020-01-29 00:00:00,2020-01-29 00:00:00,2,5750,"""CADDY""","""2""",2026-01-29 00:00:00,"""06""",false
2023-11-29 00:00:00,2023-11-29 00:00:00,1,2600,"""Mustang Mach-E…","""5""",2027-11-29 00:00:00,"""06""",true
2020-01-27 00:00:00,2020-01-27 00:00:00,2,5750,"""CADDY""","""2""",2024-01-27 00:00:00,"""01""",false
2020-05-19 00:00:00,2020-05-19 00:00:00,1,5750,"""CRAFTER""","""2""",2024-05-19 00:00:00,"""01""",false
…,…,…,…,…,…,…,…,…
2021-10-21 00:00:00,2021-10-21 00:00:00,2,3900,"""GLE 350 de 4MA…","""2,5""",2025-10-21 00:00:00,"""03""",false
2023-09-01 00:00:00,2023-09-01 00:00:00,0,320,"""Adora 573 PT""",null,null,"""01""",null
2020-12-29 00:00:00,2020-12-29 00:00:00,1,4000,"""MG ZS EV""","""5""",2024-12-29 00:00:00,"""02""",true


In [13]:
parquet_file_path = "kjoretoyinfo_preppet.parquet"


df_selected.write_parquet(parquet_file_path)